# Experiment 05: Data augmentation

In this experiment we will study how different band configurations affect the training process and end results.

### Experiment variations:

- E03-4: The 4 MS WorldView-2 bands that are also available in GeoEye-1 (MS1 array RGB+NIR1)
 - In this variation we will also validate performance on the GeoEye-1 validation set
- E03-3: Only the 3 RGB bands from WorldView-2
 - Also validated on the GeoEye-1 validation set
 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *
from modules.evaluation import *

from modules.logging import *
from modules.train import *

import time

# Check GPUs and enable dynamic GPU memory use:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e05-8', 'e05-6', 'e05-4', 'e05-3']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[2]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRE_BUILD = True          # Step 1 of the training process
PRETRAIN = False          # Step 1 of the training process
GAN_BUILD = True          # Step 1 of the training process
GAN_TRAIN = False         # Step 2 of the training process
PRE_EVALUATE_LAST = True
GAN_EVALUATE_LAST = False
PRE_EVALUATE_HISTORY = False
GAN_EVALUATE_HISTORY = False

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e05'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 2.0, 
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.127841.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
    NEW_PIXEL_SIZE_PAN = 0.5
    RESIZE_DIR = DATA_PATH + '-resized'
    
# Data augmentation
AUGMENT_FLIP = True # both up/down and left/right flips
AUGMENT_ROTATE = True # 90 degree rotations
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e05-8': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e05-6': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e05-4': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}, 
               'e05-3': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e05-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e05-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e05-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e05-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e05-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                         'e05-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                   if k not in ['Coastal', 'NIR2']}, 
                         'e05-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                   if k in ['Blue', 'Green', 'Red', 'NIR']},
                         'e05-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e05-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e05-8': {None: None},                                   # not enough bands in GE01
                         'e05-6': {None: None},                                    # not enough bands in GE01
                         'e05-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                         'e05-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e05-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 16 # Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 0     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 1
VAL_N_BATCHES_SAVE = 1

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 0
# Proportion of val batches that will go through ma and niqe metric calculation
# MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
MA_NIQE_PROPORTION = 1  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_DIRS = {'e05-4': LOGS_EXP_DIR + '/models/' + 'e05-4-pre_20210319-092729/'
                        }
PRETRAIN_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIRS[EXPERIMENT]
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_DIR + EXPERIMENT + '-pre-400.h5'

# Path to the gan-training weights that will be 
GAN_WEIGHTS_DIRS = {'e05-4': LOGS_EXP_DIR + '/models/' + 'e05-4-gan_20210321-101445/'
                   }
GAN_WEIGHTS_DIR = GAN_WEIGHTS_DIRS[EXPERIMENT]
GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-399.h5'
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = False
METRIC_NIQE = False
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
#################################################################################################################

### EVALUTAION ##################################################################################################
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    METRIC_MA = True
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False

EVAL_STEPS_PER_EPOCH = 'all'
EVAL_N_EPOCHS = 400
#EVAL_SENSOR = 'WV02'
EVAL_PER_IMAGE = True
    
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        
#if PRE_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 1
#    EVAL_PREFIX = EXPERIMENT + '-pre-'
#elif GAN_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = GAN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 0
#    EVAL_PREFIX = EXPERIMENT + '-gan-'
    
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 22, 'val': 19, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4, 'NIR': 6}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2, 'NIR': 3}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: True
NIQE: True
Perceptual Index (PI): True


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [3]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 129221, 'val': 8113, 'test': 9293}
Total number of tiles: 146627


### 2.3 Tile generation to disk

In [4]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [5]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [6]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 341.3 and sd 128.4 from json file @ data/toulon-laspezia-tiles/e05/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [7]:
SHUFFLE_BUFFER_SIZE = {'train': n_tiles['train'],  # 100
                       'val': n_tiles['val'],  # 100
                       'test': n_tiles['test']}  # 100

train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-' 
                                                  + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test],
                                   augment_flip=True,
                                   augment_rotate=True
                                  )
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.2006480509994506
Output (float) min, max: -0.2000617970682984 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [8]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [9]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [10]:
if PRE_BUILD:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [11]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [12]:
if GAN_BUILD:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH)

Starting matlab.engine ...
matlab.engine started


## 7. GAN training

In [13]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

## 8. Evaluation

### 8.1 Data input pipelines for final evaluation

The pipeline is modified to include the file paths of the tiles/patches so that it is possible to log performance metrics for individual files and by extension for individual satellite images.

#### 8.1.1 Validation set

In [14]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=False,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_filepath_cache'), 
                                    repeat=False, 
                                    shuffle=False, 
                                    shuffle_buffer_size=0, #SHUFFLE_BUFFER_SIZE[train_val_test], 
                                    include_file_paths=True)
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


### 8.2 Evaluate last epoch

In [15]:
## OVERRIDE LAST!
#GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-149.h5'
FINAL_EPOCH_NAME = 'final_epoch-'
#FINAL_EPOCH_NAME = 'epoch_149-'
#SENSORS = {'val': ['WV02']}

if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    val_or_test = 'val'
    
    if PRE_EVALUATE_LAST and not GAN_EVALUATE_LAST:
        PRE_GAN = ['pre']
    if not PRE_EVALUATE_LAST and GAN_EVALUATE_LAST:
        PRE_GAN = ['gan']

    print(PRE_GAN, SENSORS[val_or_test])
    for pre_gan in PRE_GAN:
        
        for sensor in SENSORS[val_or_test]:
            # if sensor == 'GE01':
            #     continue
                
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
            else:
                gan_model.G.load_weights(GAN_WEIGHTS_PATH)

            print(pre_gan, sensor)
            start = time.time()
            results_df = esrgan_evaluate(model=gan_model, 
                                         dataset=ds_val[sensor], 
                                         steps='all', 
                                         per_image=True, 
                                         write_csv=True,
                                         csv_path=str(LOGS_EXP_DIR + '/csv/' + FINAL_EPOCH_NAME 
                                                      + pre_gan + '-' + val_or_test + '-' + sensor + '.csv'), 
                                         verbose=1
                                        )
            end = time.time()
            print(str((end - start) / 60), 'minutes')

['pre'] ['WV02', 'GE01']
pre WV02
Computed 8 images in  132.49410772323608 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 36.124629974365234, 'SSIM': 0.9352430701255798, 'Ma': 2.3832287788391113, 'NIQE': 7.879097938537598, 'PI': 7.747934341430664}
Computed 8 images in  124.98028016090393 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 40.576698303222656, 'SSIM': 0.9837294220924377, 'Ma': 3.1950042247772217, 'NIQE': 10.791642189025879, 'PI': 8.798318862915039}
Computed 8 images in  124.66003966331482 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 28.776044845581055, 'SSIM': 0.8703519701957703, 'Ma': 2.9075510501861572, 'NIQE': 6.495738506317139, 'PI': 6.794094085693359}
Computed 8 images in  124.889786481

Computed 8 images in  121.8490903377533 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 38.488990783691406, 'SSIM': 0.9618426561355591, 'Ma': 2.9021263122558594, 'NIQE': 7.017320156097412, 'PI': 7.0575971603393555}
Computed 8 images in  122.232980966568 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 35.619110107421875, 'SSIM': 0.8420394062995911, 'Ma': 2.604743719100952, 'NIQE': 7.5968217849731445, 'PI': 7.496038913726807}
Computed 8 images in  122.32190561294556 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 31.69921875, 'SSIM': 0.8604134917259216, 'Ma': 2.7686209678649902, 'NIQE': 6.759611129760742, 'PI': 6.995494842529297}
Computed 8 images in  122.31196355819702 seconds
Last image: {'G_pixel_loss': 0

Computed 8 images in  121.39872479438782 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 32.08380126953125, 'SSIM': 0.9075639843940735, 'Ma': 3.174733877182007, 'NIQE': 6.965622901916504, 'PI': 6.895444393157959}
Computed 8 images in  122.55456161499023 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 57.892581939697266, 'SSIM': 0.9990203380584717, 'Ma': 2.7289493083953857, 'NIQE': 20.207435607910156, 'PI': 13.739242553710938}
Computed 8 images in  122.95590233802795 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 33.03352737426758, 'SSIM': 0.8709732890129089, 'Ma': 2.702169418334961, 'NIQE': 6.7395124435424805, 'PI': 7.01867151260376}
Computed 8 images in  122.28106784820557 seconds
Last image: {'G_pixel_l

Computed 8 images in  121.65854001045227 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 32.9665641784668, 'SSIM': 0.8033536672592163, 'Ma': 2.723257303237915, 'NIQE': 6.8136396408081055, 'PI': 7.045191287994385}
Computed 8 images in  122.09466028213501 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 36.0821418762207, 'SSIM': 0.8857542276382446, 'Ma': 2.7831740379333496, 'NIQE': 6.628965854644775, 'PI': 6.922895908355713}
Computed 8 images in  121.10131359100342 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 27.173614501953125, 'SSIM': 0.7486514449119568, 'Ma': 2.8816323280334473, 'NIQE': 8.933971405029297, 'PI': 8.026169776916504}
Computed 8 images in  122.94655203819275 seconds
Last image: {'G_pixel_los

Computed 8 images in  121.35874795913696 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 34.802860260009766, 'SSIM': 0.8391427993774414, 'Ma': 2.8326916694641113, 'NIQE': 7.381187438964844, 'PI': 7.274248123168945}
Computed 8 images in  121.16822624206543 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 25.806629180908203, 'SSIM': 0.668491005897522, 'Ma': 2.7680647373199463, 'NIQE': 7.151900291442871, 'PI': 7.191917896270752}
Computed 8 images in  121.01027917861938 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 27.565034866333008, 'SSIM': 0.8553792834281921, 'Ma': 2.6648049354553223, 'NIQE': 6.380100727081299, 'PI': 6.857647895812988}
Computed 8 images in  122.72116756439209 seconds
Last image: {'G_pixel_

...

Computed 8 images in  121.20925521850586 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 29.974302291870117, 'SSIM': 0.8036572337150574, 'Ma': 2.498905658721924, 'NIQE': 6.9477152824401855, 'PI': 7.224404811859131}
Computed 8 images in  121.8931143283844 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 28.28080940246582, 'SSIM': 0.6939656734466553, 'Ma': 2.720973253250122, 'NIQE': 7.30388069152832, 'PI': 7.291453838348389}
Computed 8 images in  121.32369327545166 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 47.25993728637695, 'SSIM': 0.996450662612915, 'Ma': 2.1070358753204346, 'NIQE': 10.972785949707031, 'PI': 9.43287467956543}
Computed 8 images in  120.95186805725098 seconds
Last image: {'G_pixel_loss'

Computed 8 images in  121.75203967094421 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 30.3240966796875, 'SSIM': 0.8811591863632202, 'Ma': 3.335808277130127, 'NIQE': 7.837491035461426, 'PI': 7.25084114074707}
Computed 8 images in  120.90598034858704 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 35.091949462890625, 'SSIM': 0.9752825498580933, 'Ma': 2.9197118282318115, 'NIQE': 11.609654426574707, 'PI': 9.344970703125}
Computed 8 images in  121.03724431991577 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 27.684356689453125, 'SSIM': 0.6206395626068115, 'Ma': 2.5547971725463867, 'NIQE': 7.135372161865234, 'PI': 7.290287494659424}
Computed 8 images in  120.59407234191895 seconds
Last image: {'G_pixel_loss'

Computed 8 images in  120.88778448104858 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 28.321680068969727, 'SSIM': 0.8892325162887573, 'Ma': 3.0416574478149414, 'NIQE': 7.761965274810791, 'PI': 7.360154151916504}
Computed 8 images in  121.03282475471497 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 28.60675621032715, 'SSIM': 0.5991930365562439, 'Ma': 2.639000177383423, 'NIQE': 7.36794376373291, 'PI': 7.364471912384033}
Computed 8 images in  121.26458692550659 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 27.34789276123047, 'SSIM': 0.555206835269928, 'Ma': 2.578380823135376, 'NIQE': 7.104402542114258, 'PI': 7.2630109786987305}
Computed 8 images in  121.28128147125244 seconds
Last image: {'G_pixel_loss

Computed 8 images in  121.47810888290405 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 29.542890548706055, 'SSIM': 0.8965811729431152, 'Ma': 2.761812210083008, 'NIQE': 11.511123657226562, 'PI': 9.374655723571777}
Computed 8 images in  121.30280542373657 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 26.428829193115234, 'SSIM': 0.8547634482383728, 'Ma': 2.856905460357666, 'NIQE': 7.1762495040893555, 'PI': 7.159671783447266}
Computed 8 images in  121.01594018936157 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 22.218219757080078, 'SSIM': 0.6714279651641846, 'Ma': 2.740280866622925, 'NIQE': 6.742063522338867, 'PI': 7.000891208648682}
Computed 8 images in  120.26229691505432 seconds
Last image: {'G_pixel_

Computed 8 images in  120.32787370681763 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 24.003795623779297, 'SSIM': 0.6032023429870605, 'Ma': 2.607388973236084, 'NIQE': 8.940613746643066, 'PI': 8.16661262512207}
Computed 8 images in  119.49234652519226 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 26.09212303161621, 'SSIM': 0.6462915539741516, 'Ma': 2.555948257446289, 'NIQE': 8.198136329650879, 'PI': 7.821094036102295}
Computed 8 images in  120.62558960914612 seconds
Last image: {'G_pixel_loss': 0.0, 'G_perceptual_loss': 0.0, 'G_generator_loss': 0.0, 'G_loss_total': 0.0, 'D_loss_total': 0.0, 'PSNR': 27.106487274169922, 'SSIM': 0.8594817519187927, 'Ma': 3.1098084449768066, 'NIQE': 8.183379173278809, 'PI': 7.536785125732422}
Computed 8 images in  122.58143997192383 seconds
Last image: {'G_pixel_los

### 8.3 Evaluate every kth epoch

In [16]:
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    val_or_test = 'val'
    
    # Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
    if METRIC_MA:
        PRE_GAN = ['gan']
    else:
        PRE_GAN = ['pre', 'gan']

    for pre_gan in PRE_GAN:
        for sensor in SENSORS[val_or_test]:
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                model_weights_dir = PRETRAIN_WEIGHTS_DIR
                eval_first_step = 1
                eval_prefix = EXPERIMENT + '-pre-'
            else:
                model_weights_dir = GAN_WEIGHTS_DIR
                eval_first_step = 0
                eval_prefix = EXPERIMENT + '-gan-'

            esrgan_epoch_evaluator(gan_model,
                                   model_weights_dir=model_weights_dir,
                                   model_weight_prefix=eval_prefix,
                                   dataset=ds_val[sensor],
                                   n_epochs=EVAL_N_EPOCHS,
                                   first_epoch=eval_first_step,
                                   steps_per_epoch=EVAL_STEPS_PER_EPOCH,
                                   k_epoch=25,
                                   csv_dir=LOGS_EXP_DIR + '/csv/' + val_or_test + '-' + sensor, 
                                       per_image=EVAL_PER_IMAGE, 
                                   verbose=0)

### 8.4 Comparison plots